## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata_df = pd.read_csv(mouse_metadata_path)
study_results_df = pd.read_csv(study_results_path)



In [2]:
#view mouse data
mouse_metadata_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16


In [3]:
#view study results
study_results_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0
1,f932,0,45.0,0
2,g107,0,45.0,0
3,a457,0,45.0,0
4,c819,0,45.0,0


In [4]:
#look at value counts to find duplicate mice
study_results_df['Mouse ID'].value_counts()


g989    13
z578    10
j365    10
g288    10
a262    10
        ..
n482     1
t573     1
b447     1
l872     1
h428     1
Name: Mouse ID, Length: 249, dtype: int64

In [5]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
find_dup_mouse_id = study_results_df.loc[study_results_df.duplicated(subset=['Mouse ID', 'Timepoint',]),'Mouse ID'].unique()
print(find_dup_mouse_id)

['g989']


In [6]:
# Optional: Get all the data for the duplicate mouse ID. duplicate is mouse g989.
mouse_metadata_df.loc[mouse_metadata_df['Mouse ID'] == 'g989']

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
107,g989,Propriva,Female,21,26


In [7]:
# Optional: Get all the data for the duplicate mouse ID. -part 2
study_results_df.loc[study_results_df['Mouse ID'] == 'g989']

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
107,g989,0,45.000000,0
137,g989,0,45.000000,0
329,g989,5,48.786801,0
360,g989,5,47.570392,0
620,g989,10,51.745156,0
681,g989,10,49.880528,0
815,g989,15,51.325852,1
869,g989,15,53.442020,0
950,g989,20,55.326122,1
1111,g989,20,54.657650,1


In [8]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.

clean_study_results_df = study_results_df[study_results_df['Mouse ID'].isin(find_dup_mouse_id)==False]
clean_mouse_data_df = mouse_metadata_df[mouse_metadata_df['Mouse ID'].isin(find_dup_mouse_id)==False]


In [9]:
#combine the dataframes using Mouse ID
combined_mouse_data_df = pd.merge(clean_study_results_df, clean_mouse_data_df, on = "Mouse ID")
combined_mouse_data_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22


In [10]:
# Checking the number of mice in the DataFrame. = all good data now, mice = 248
combined_mouse_data_df['Mouse ID'].value_counts()

z578    10
j755    10
y769    10
g288    10
a262    10
        ..
u153     1
n482     1
t573     1
b447     1
h428     1
Name: Mouse ID, Length: 248, dtype: int64

## Summary Statistics

In [11]:
# Generate a summary statistics table of mean, median, variance, standard deviation, 
#and SEM of the tumor volume for each regimen
# This method is the most straighforward, creating multiple series and putting them all together at the end.
# Generate a summary statistics table by chaining .groupby(), .agg() and .round() methods

tumorvol_drug_stats_df = combined_mouse_data_df.groupby('Drug Regimen').agg(\
        TumorVolume_Mean=('Tumor Volume (mm3)', np.nanmean), \
        TumorVolume_Median=('Tumor Volume (mm3)', np.nanmedian), \
        TumorVolume_Variance=('Tumor Volume (mm3)', np.nanvar), \
        TumorVolume_StandardDeviation=('Tumor Volume (mm3)', np.nanstd), \
        TumorVolume_SEM=('Tumor Volume (mm3)', st.sem)).round(2)
      
    
# Generate a summary statistics table of mean, median, variance, standard deviation, 
#and SEM of the tumor volume for each regimen

tumorvol_drug_stats_df


C:\Anaconda\lib\site-packages\numpy\core\_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Anaconda\lib\site-packages\numpy\core\_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Anaconda\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,TumorVolume_Mean,TumorVolume_Median,TumorVolume_Variance,TumorVolume_StandardDeviation,TumorVolume_SEM
Drug Regimen,,,,,
Capomulin,40.68,41.56,24.95,4.99,0.33
Ceftamin,52.59,51.78,39.29,6.27,0.47
Infubinol,52.88,51.82,43.13,6.57,0.49
Ketapril,55.24,53.70,68.55,8.28,0.60
Naftisol,54.33,52.51,66.17,8.13,0.60
Placebo,54.03,52.29,61.17,7.82,0.58
Propriva,52.32,50.45,43.85,6.62,0.54
Ramicane,40.22,40.67,23.49,4.85,0.32
Stelasyn,54.23,52.43,59.45,7.71,0.57


## Bar Plots

In [12]:
# Generate a bar plot showing the number of mice per time point for each treatment throughout the course of the study using pandas. 

In [13]:
# Generate a bar plot showing the number of mice per time point for each treatment throughout the course of the study using pyplot.

## Pie Plots

In [14]:
# Generate a pie plot showing the distribution of female versus male mice using pandas

In [15]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot

## Quartiles, Outliers and Boxplots

In [16]:
# Calculate the final tumor volume of each mouse across four of the most promising treatment regimens. Calculate the IQR and quantitatively determine if there are any potential outliers. 


In [17]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest

## Line and Scatter Plots

In [18]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [19]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen



## Correlation and Regression

In [20]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
